# Les requêtes de modification

* Auteurs/trices : Rémi Leduc, Léo Rouger, Clément Caillard

Ce chapitre traite des différentes requêtes de modification (insertion, modification, suppression)

Le fichier que vous devez modifier pour ce chapitre est `mongo_book/content/06_modif.md`.